# Module 8: 13 - Anatomy of an AI Agent - Enabling Tool Selection via Prompting
---------------------------------------------------------------------------------
In this lesson, we will support LLMs that do not have function calling capabilities by using a basic prompt. We will pass the tools and their details, such as function signatures, to the LLM. This will allow the LLM to output a structured response containing the tool details in JSON format, similar to how OpenAI Function Calling works.

## Objectives
* Support LLMs that do not have built-in function calling capabilities.
* Use prompts to pass tool details and function signatures to the LLM.
* Generate structured responses with tool details in JSON format.
* Ensure seamless tool execution by interpreting the LLM's structured responses.

## What this session covers:
* Defining the current agent structure, including the LLM Client and short-term memory.
* Creating prompts to pass tool details and function signatures to the LLM.
* Interpreting the LLM's structured responses for tool execution.
* Implementing a mechanism to parse and validate the LLM's output.
* Integrating and testing the enhanced Agent with LLMs that lack function calling capabilities.

## Install Libraries

In [2]:
#! pip install openai

## Define Current Agent Structure

### LLM Client

In [2]:
from typing import Dict, Any, List
import openai

class OpenAIChatCompletion:
    """Interacts with OpenAI's API for chat completions."""
    def __init__(self, model: str, api_key: str = None, base_url: str = None):
        self.client = openai.OpenAI(api_key=api_key, base_url=base_url)
        self.model = model

    def generate(self, messages: List[str], tools: List[Dict[str, Any]] = None, **kwargs) -> Dict[str, Any]:
        """Generates a response from OpenAI's API."""
        params = {'messages': messages, 'model': self.model, 'tools': tools, **kwargs}
        response = self.client.chat.completions.create(**params)
        return response.choices[0].message

### Short-Term Memory

In [3]:
from typing import List, Dict

class ChatMessageMemory:
    """Manages conversation context."""
    
    def __init__(self):
        self.messages = []
    
    def add_message(self, message: Dict):
        """Add a message to memory."""
        self.messages.append(message)
    
    def add_messages(self, messages: List[Dict]):
        """Add multiple messages to memory."""
        for message in messages:
            self.add_message(message)
    
    def get_messages(self) -> List[Dict]:
        """Retrieve all messages."""
        return self.messages.copy()
    
    def reset_memory(self):
        """Clear all messages."""
        self.messages = []

### Agent Tool

In [4]:
from pydantic import BaseModel, ValidationError
from typing import Callable, Type
from inspect import signature

class AgentTool:
    """Encapsulates a Python function with Pydantic validation."""
    def __init__(self, func: Callable, args_model: Type[BaseModel]):
        self.func = func
        self.args_model = args_model
        self.name = func.__name__
        self.description = func.__doc__ or self.args_schema.get('description', '')

    def to_openai_function_call_definition(self) -> dict:
        """Converts the tool to OpenAI Function Calling format."""
        schema_dict = self.args_schema
        description = schema_dict.pop("description", "")
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": description,
                "parameters": schema_dict
            }
        }

    @property
    def args_schema(self) -> dict:
        """Returns the tool's function argument schema as a dictionary."""
        schema = self.args_model.model_json_schema()
        schema.pop("title", None)
        return schema

    def validate_json_args(self, json_string: str) -> bool:
        """Validate JSON string using the Pydantic model."""
        try:
            validated_args = self.args_model.model_validate_json(json_string)
            return isinstance(validated_args, self.args_model)
        except ValidationError:
            return False

    def run(self, *args, **kwargs) -> Any:
        """Execute the function with validated arguments."""
        try:
            # Handle positional arguments by converting them to keyword arguments
            if args:
                sig = signature(self.func)
                arg_names = list(sig.parameters.keys())
                kwargs.update(dict(zip(arg_names, args)))

            # Validate arguments with the provided Pydantic schema
            validated_args = self.args_model(**kwargs)
            return self.func(**validated_args.model_dump())
        except ValidationError as e:
            raise ValueError(f"Argument validation failed for tool '{self.name}': {str(e)}")
        except Exception as e:
            raise ValueError(f"An error occurred during the execution of tool '{self.name}': {str(e)}")

    def __call__(self, *args, **kwargs) -> Any:
        """Allow the AgentTool instance to be called like a regular function."""
        return self.run(*args, **kwargs)

### Agent Tool Decorator

In [5]:
from typing import Callable, Optional, Type
from pydantic import BaseModel

def check_docstring(func: Callable):
    """Ensure the function has a docstring."""
    if not func.__doc__:
        raise ValueError(f"Function '{func.__name__}' must have a docstring.")

def Tool(func: Optional[Callable] = None, *, args_model: Type[BaseModel]) -> AgentTool:
    """Decorator to wrap a function with an AgentTool instance."""
    def decorator(f: Callable) -> AgentTool:
        check_docstring(f)
        return AgentTool(f, args_model=args_model)
    return decorator(func) if func else decorator

### Agent Tool Executor

In [6]:
from typing import Any, Dict, List, Optional
import json

class AgentToolExecutor:
    """Manages tool registration and execution."""
    
    def __init__(self, tools: Optional[List[AgentTool]] = None):
        self.tools: Dict[str, AgentTool] = {}
        if tools:
            for tool in tools:
                self.register_tool(tool)
    
    def register_tool(self, tool: AgentTool):
        """Registers a tool."""
        if tool.name in self.tools:
            raise ValueError(f"Tool '{tool.name}' is already registered.")
        self.tools[tool.name] = tool
      
    def execute(self, tool_name: str, tool_args: str) -> Any:
        """Executes a tool by name with given arguments."""
        print(f"Checking if {tool_name} exists..")
        tool = self.tools.get(tool_name)
        if not tool:
            raise ValueError(f"Tool '{tool_name}' not found.")
        try:
            print(f"Validating {tool_name} suggested args {tool_args}")
            if tool.validate_json_args(tool_args):
                tool_args_dict = json.loads(tool_args)
                print(f"Executing {tool_name} with args: {tool_args}")
                return tool.run(**tool_args_dict)
            else:
                raise ValueError(f"Error validating tool '{tool_name}' arguments.")
        except Exception as e:
            raise ValueError(f"Error executing tool '{tool_name}': {e}") from e
    
    def get_tool_names(self) -> List[str]:
        """Returns a list of all registered tool names."""
        return list(self.tools.keys())
    
    def get_tool_details(self) -> str:
        """Returns details of all registered tools."""
        tools_info = [f"{tool.name}: {tool.description} Args schema: {tool.args_schema['properties']}" for tool in self.tools.values()]
        return '\n'.join(tools_info)

### Agent Base

In [22]:
from typing import Dict, List, Optional

class Agent:
    """Integrates LLM client, tools, memory, and manages tool executions."""
    
    def __init__(self, llm_client, system_message: Dict[str, str], max_iterations: int = 10, tools: Optional[List[AgentTool]] = None):
        self.llm_client = llm_client
        self.executor = AgentToolExecutor()
        self.memory = ChatMessageMemory()
        self.system_message = system_message
        self.max_iterations = max_iterations
        self.tool_history = []
        self.function_calls = None
        
        # Register and convert tools
        if tools:
            for tool in tools:
                self.executor.register_tool(tool)
            self.function_calls = [tool.to_openai_function_call_definition() for tool in tools]

    def run(self, user_message: Dict[str, str]):
        """Generates responses, manages tool calls, and updates memory."""
        self.memory.add_message(user_message)

        for _ in range(self.max_iterations):
            chat_history = [self.system_message] + self.memory.get_messages() + self.tool_history
            print(f"chat history sent: {chat_history}")
            response = self.llm_client.generate(chat_history, tools=self.function_calls)

            if self.parse_response(response):
                continue
            else:
                self.memory.add_message(response)
                self.tool_history = []
                return response

    def parse_response(self, response) -> bool:
        """Executes tool calls suggested by the LLM and updates tool history."""
        if response.tool_calls:
            self.tool_history.append(response)
            for tool in response.tool_calls:
                tool_name = tool.function.name
                tool_args = tool.function.arguments
                try:
                    execution_results = self.executor.execute(tool_name, tool_args)
                    self.tool_history.append({
                        "role": "tool",
                        "tool_call_id": tool.id,
                        "name": tool_name,
                        "content": str(execution_results)
                    })
                except Exception as e:
                    raise ValueError(f"Execution error in tool '{tool_name}': {e}") from e
            return True
        return False

## Testing Tool Selection via Prompt

### Initialize Client

In [23]:
# API from environment variable
# import os
# api_key = os.getenv("OPENAI_API_KEY"))

api_key=""

client = OpenAIChatCompletion(
    base_url='https://api.openai.com/v1',
    model='gpt-4o',
    api_key=api_key
)

### Define System messages

In [24]:
# Define the system message
system_message = {"role": "system", "content": "You are a weather assistant."}

### Initialize Agent

In [39]:
# Initialize the Agent with the LLM client and system message
agent = Agent(llm_client=client, system_message=system_message)

### Craft Prompt - User Message

In [54]:
USER_PROMPT = """
You have access to the following tools:

search(topic: str) - Use it to search information.
greet(person: str) - Use it to greet a person.

Respond to the user with the right tool and input whenever is needed.
When responding to the user, provide only ONE tool per $JSON_BLOB, as shown
in the example below delimited by triple backticks:

```
{{
    "name": $TOOL_NAME,
    "arguments": $INPUT
}}
```

User input: Hello, this is roberto!
"""

# Define a user message
user_message = {"role": "user", "content": USER_PROMPT}

### Send a User Message

In [55]:
# Generate a response using the agent
response = agent.run(user_message)
response

chat history sent: [{'role': 'system', 'content': 'You are a weather assistant.'}, {'role': 'user', 'content': '\nYou have access to the following tools:\n\nsearch(topic: str) - Use it to search information.\ngreet(person: str) - Use it to greet a person.\n\nRespond to the user with the right tool and input whenever is needed.\nWhen responding to the user, provide only ONE tool per $JSON_BLOB, as shown\nin the example below delimited by triple backticks:\n\n```\n{{\n    "name": $TOOL_NAME,\n    "arguments": $INPUT\n}}\n```\n\nUser input: Hello, this is roberto!\n'}, ChatCompletionMessage(content='```\n{{\n    "name": "greet",\n    "arguments": "Roberto"\n}}\n```', role='assistant', function_call=None, tool_calls=None), {'role': 'user', 'content': '\nYou have access to the following tools:\n\nsearch(topic: str) - Use it to search information.\ngreet(person: str) - Use it to greet a person.\n\nRespond to the user with the right tool and input whenever is needed.\nWhen responding to the u

ChatCompletionMessage(content='```\n{{\n    "name": "greet",\n    "arguments": "roberto"\n}}\n```', role='assistant', function_call=None, tool_calls=None)

### What about JSON within Text?

Instructing an LLM to output JSON can result in mixed explanatory text and structured responses. When this happens, we can use regex patterns to extract the JSON embedded within the text.

For parsing complex nested JSON structures, Python's re module isn't sufficient since it lacks nested pattern matching support. The regex library, however, allows recursion with patterns like r'\{(?:[^{}]|(?R))*\}', enabling effective parsing of deeply nested JSON data. (https://stackoverflow.com/a/54235803).

In [59]:
import regex
import json

def parse_nested_json(text):
    # Unescape backslashes
    text = text.replace('\\\\n', '\\n').replace('\\n', '\n').replace('\\\'', '\'').replace('\\\\', '\\')
    # Replace double curly braces with single curly braces
    text = text.replace('{{', '{').replace('}}', '}')
    pattern = regex.compile(r'\{(?:[^{}]|(?R))*\}') # Supports nested structures
    match = pattern.search(text)
    if match:
        try:
            return json.loads(match.group())
        except json.JSONDecodeError:  # Corrected to use json.JSONDecodeError
            pass
    return None  # No valid JSON found or parsing error

In [60]:
json_within_text = f"Here is its {response}"
json_within_text

'Here is its ChatCompletionMessage(content=\'```\\n{{\\n    "name": "greet",\\n    "arguments": "roberto"\\n}}\\n```\', role=\'assistant\', function_call=None, tool_calls=None)'

In [61]:
parse_nested_json(json_within_text)

{'name': 'greet', 'arguments': 'roberto'}